In [22]:
from dotenv import load_dotenv
load_dotenv()
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings 
from llama_index.core.tools import QueryEngineTool
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.multi_modal_llms.ollama import OllamaMultiModal

In [34]:
# settings
Settings.llm = OllamaMultiModal(model="llava", temperature=0)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5") 

In [36]:
# function tools
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

In [37]:

documents = SimpleDirectoryReader("./Data").load_data()
index = MultiModalVectorStoreIndex.from_documents(documents)
print(index)

query_engine = index.as_query_engine()

In [52]:
response = query_engine.query("In Latency-Aware Resource Allocation in which episode does Exploration happen?")
print(response)

 The exploration phase of Latency-Aware Resource Allocation occurs during the initial stages of training, as indicated by the SHAP value for URLLC users experiencing a low SNR state (SHAP value =−1.77) and the relatively low allocation of PRBs per slice (8.99PRBs). 


In [39]:
# rag pipeline as a tool
ml_ops_tool = QueryEngineTool.from_defaults(
    query_engine, 
    name="ML OPs",
    description="A RAG engine with some basic facts about the ML Ops."
)

In [40]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, ml_ops_tool], verbose=True)

In [48]:
response = agent.chat("Can you add years of experience for DEVOPs for a ML engineering consultant and an AI architect/consultant")

> Running step 569c0da2-184f-4034-8942-24a936f6125f. Step input: Can you add years of experience for DEVOPs for a ML engineering consultant and an AI architect/consultant
Thought: (Implicit) I can answer without any more tools!
Answer:  It is not possible to determine the exact number of years of experience required for a specific job title, as these requirements can vary depending on the organization and industry. Additionally, I do not have access to information about the current market or trends in the field of DevOps, which would be necessary to make an accurate assessment. 


In [ ]:
print(response)